In [1]:
!pip install sklearn progressbar2

     |████████████████████████████████| 6.8 MB 3.7 MB/s eta 0:00:01     |█████████████████████████▎      | 5.4 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 5.0 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=76421888750e02c158b6241030b0f613575f4444911c48f02c94b1d08b864d66
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random
import math
from sklearn.metrics import classification_report, confusion_matrix
import progressbar

In [3]:
GB = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
#The variable GB is the memory size you want to use.
config = [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*GB))]
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], config)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
np.set_printoptions(threshold=40*40)


In [4]:
def load_data(res, source_labels, dataset='train', size=1., verbosity = False):
    data = np.zeros((0, res, res, res))
    labels = []
    for i, label in enumerate(source_labels):
        filename = './output/' + label + '/output-' + dataset + '-' + str(res) + '.h5'
        file = h5py.File(filename, 'r')
        tensor_len = len(file['tensor'])
        picked_ids = random.sample(range(tensor_len), math.ceil(tensor_len * size))
        if verbosity:
            print(filename)
            print('Picking ' + str(math.ceil(tensor_len * size)) + ' from ' + str(len(file['tensor'])) )
        data = np.concatenate((data, [file['tensor'][index] for index in picked_ids]))
        labels = np.concatenate((labels, np.full(len(picked_ids), i)))
        file.close()
        gc.collect()
    rng_state = np.random.get_state()
    np.random.shuffle(data)
    np.random.set_state(rng_state)
    np.random.shuffle(labels)
    return (data, labels)


In [ ]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(res, res, res)),
        keras.layers.Dense(res*2, activation='relu'),
        keras.layers.Dense(res/2, activation='sigmoid'),
        keras.layers.Dropout(.2, input_shape=(2,)),
        keras.layers.Dense(40, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes):
    return data
model_type = 'dense'
prepare_model(24, 20, 1)

In [5]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Conv3D(
                res, 
                kernel_size=(3), 
                activation='relu', 
                kernel_initializer='he_uniform', 
                data_format="channels_last",
                input_shape=(res, res, res, num_classes)
            ),        
        keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(res/2, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes):
    return keras.utils.to_categorical(data, num_classes)
model_type = 'conv3d'
#prepare_model(24, 10, 1)

In [6]:
def train(model, training, training_labels, res, num_classes, verbosity = 0):
    log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    if verbosity == 1:
        print("Log file " + log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    validation_split = 0.1
    epochs = 10 * num_classes
    
#    model.fit(
#        keras.utils.to_categorical(training, num_clases), 
#        training_labels, 
#        epochs=epochs)
    
    model.fit(
        prepare_data(training, num_classes), 
        training_labels, 
        epochs=epochs,
        verbose=verbosity,
        callbacks=[tensorboard_callback],
        validation_split=validation_split
    )

In [7]:
#Confusion Matrix and Classification Report
def print_confusion_matrix(test, test_labels, base_labels):
    Y_pred = model.predict_generator(prepare_data(test, len(base_labels)), len(test))
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    print(confusion_matrix(test_labels, y_pred))
    print('Classification Report')
    print(classification_report(test_labels, y_pred, target_names=base_labels))

In [8]:
#base_labels = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
all_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 
                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
                 'vase', 'wardrobe', 'xbox']


In [ ]:
subclases = [10, 20, 30, 40]
resolutions = [24]#, 32]#, 48]
num_classes = 2

for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        base_labels = [all_labels[i]] + [all_labels[j]]
        print(i, j, base_labels)
        for n, res in enumerate(resolutions):
            training, training_labels = load_data(res, base_labels, 'train', 1, False)
            model = prepare_model(res, num_classes, 0)
            model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'
            train(model, training, training_labels, res, num_classes, 0)
            model.save(model_name)
            test, test_labels = load_data(res, base_labels, 'test', 1.0, False)
            test_loss, test_acc = model.evaluate(
                prepare_data(test, num_classes),  
                test_labels, 
                verbose=2
            )
            print('\nTest accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':', test_acc)
            print_confusion_matrix(test, test_labels, all_labels)
            gc.collect()          


In [9]:
res = 24
model_type = 'conv3d'
test, test_labels = load_data(res, all_labels, 'test', 1.0, False)


In [10]:
def real_index(prediction, i, j):
    return j if prediction else i

def prediction_for(i, j, data):
    model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
    model = keras.models.load_model(model_name)
    prediction = model.predict(data)
    return [real_index(p, i, j) for p in list(map(np.argmax, prediction))]


In [ ]:
print(test_labels[0])
test[0].shape
to_test = np.expand_dims(test[0], axis=0)
#model.predict(to_test)

In [ ]:
prediction = [x[:] for x in [[-1] * 40] * 40]
for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        prediction[i][j] = prediction_for(i, j, to_test)
        print(prediction[i][j])


In [ ]:
i = 0
j = 5

model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
model = keras.models.load_model(model_name)
#print(model.summary())
prediction = model.predict(test)
prediction

In [11]:
np.set_printoptions(threshold=40*40*40)

def real_index(prediction, i, j):
    return j if prediction else i

def prediction_for(i, j, data):
    model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
    model = keras.models.load_model(model_name)
    #print(model.summary())
    prediction = model.predict(prepare_data(data, 2))
    return [real_index(p, i, j) for p in list(map(np.argmax, prediction))]



In [12]:
tf.get_logger().setLevel('ERROR')

In [13]:
prediction = [x[:] for x in [[-1] * 40] * 40]
for i in range(len(all_labels)):
    bar2 = progressbar.ProgressBar(max_value=len(all_labels) - (i + 1))
    for j in range(i + 1, len(all_labels)):
        bar2.update(j - (i + 1))
        prediction[i][j] = prediction_for(i, j, test)


N/A% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

In [23]:
p = [None] * len(test)
bar = progressbar.ProgressBar(max_value=len(test))
for e in range(len(test)):
    bar.update(e) 
    p[e] = []
    for i in range(40):
        for j in range(40):
            if prediction2[i][j] == -1:
                p[e].append(-1)
            else:
                p[e].append(prediction2[i][j][e])

 98% (2437 of 2468) |################### | Elapsed Time: 0:00:03 ETA:   0:00:00

In [24]:
def most_frequent(List): 
    return max(set(List), key = List.count) 
count = 0
for index in range(len(test)):
    real = test_labels[index]
    predicted = most_frequent(list(filter(lambda a: a > -1, p[index])))
    if int(real) == predicted: 
        count+=1
count

1525

In [32]:
print(all_labels[int(test_labels [0])])
p[0]

dresser


[-1,
 1,
 2,
 3,
 4,
 0,
 6,
 7,
 8,
 9,
 0,
 11,
 12,
 0,
 14,
 15,
 16,
 0,
 0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 0,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 -1,
 -1,
 2,
 3,
 4,
 5,
 1,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 14,
 15,
 16,
 1,
 1,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 1,
 29,
 1,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 -1,
 -1,
 -1,
 2,
 4,
 2,
 2,
 2,
 8,
 2,
 2,
 2,
 12,
 2,
 14,
 15,
 2,
 2,
 2,
 2,
 2,
 21,
 22,
 23,
 2,
 25,
 26,
 27,
 2,
 29,
 2,
 31,
 2,
 33,
 2,
 35,
 36,
 37,
 38,
 39,
 -1,
 -1,
 -1,
 -1,
 4,
 5,
 3,
 3,
 8,
 3,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 3,
 3,
 19,
 20,
 21,
 22,
 23,
 3,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 3,
 35,
 36,
 37,
 38,
 39,
 -1,
 -1,
 -1,
 -1,
 -1,
 4,
 4,
 4,
 8,
 4,
 10,
 4,
 4,
 4,
 14,
 15,
 16,
 4,
 4,
 4,
 4,
 21,
 22,
 23,
 4,
 4,
 4,
 27,
 4,
 4,
 30,
 4,
 32,
 33,
 4,
 35,
 36,
 37,
 38,
 39,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 5,
 5,
 8,
 5,
 10,
 5,
 12,
 5,
 14,
 15,
 1

In [17]:
np.save('prediction-conv3d.txt', prediction)

In [22]:
prediction2 = np.load('prediction-conv3d.txt.npy', allow_pickle=True)

In [ ]:
for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
        model = keras.models.load_model(model_name)
        print(model.summary())


In [ ]:
model = keras.models.load_model('models/dense-20-24')


In [ ]:
num_classes = 20
res = 24
base_labels = all_labels[:num_classes]
test, test_labels = load_data(res, base_labels, 'test', 1.0, False)


In [ ]:
test_loss, test_acc = model.evaluate(
            prepare_data(test, num_classes),  
            test_labels, 
            verbose=2
        )
print('\nTest accuracy for ' + str(num_classes) + ' classes width res ' + str(res) + ':', test_acc)
print_confusion_matrix(test, test_labels, base_labels)

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()

In [ ]:
if not len(test):
    test, test_labels = load_data(dim, base_labels, 'test', 100)
test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
p = model.predict(test)
print('\nTest accuracy:', test_acc)

In [ ]:
index = random.randint(0, len(test))

predicted_index = np.argmax(p[index])
expected = base_labels[int(test_labels[index])]
predicted = base_labels[predicted_index]
draw(test[index], expected, predicted)
index

In [ ]:
num_classes = 3
base_labels = all_labels[:num_classes]
num_classes = num_classes
res = 24
training, training_labels = load_data(res, base_labels, 'train', 1, True)


In [ ]:
print(len(training))
model = prepare_model(res, num_classes, 1)
train(model, training, training_labels, res, num_classes, 0)


In [ ]:
print(base_labels)
test, test_labels = load_data(res, base_labels, 'test', 1.0, True)
print(len(test))
test_loss, test_acc = model.evaluate(
    test,  
    test_labels, 
    verbose=2
)


In [ ]:
p = model.predict(test)
for index in range(len(p)):
    predicted_index = np.argmax(p[index])
    expected = base_labels[int(test_labels[index])]
    predicted = base_labels[predicted_index]
    if expected != predicted:
        draw(test[index], expected, predicted)
